In [2]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_geopandas, to_geopolars
from electoralyze.common.testing.region_fixture import (
    RegionMocked,
    create_fake_regions,
    read_true_geometry,
    read_true_metadata,
)
from electoralyze.region.redistribute import redistribute
from electoralyze.region.redistribute.mapping import (
    _create_intersection_area_mapping,
    _get_remaining_area,
    create_region_mapping_base,
)

In [3]:
intersection_area = _create_intersection_area_mapping(
    geometry_from=read_true_geometry("region_b"),
    geometry_to=read_true_geometry("region_c"),
)
intersection_area.pipe(print)

shape: (11, 3)
┌──────────┬──────────┬──────────┐
│ region_b ┆ region_c ┆ mapping  │
│ ---      ┆ ---      ┆ ---      │
│ str      ┆ str      ┆ f64      │
╞══════════╪══════════╪══════════╡
│ A        ┆ X        ┆ 1.21875  │
│ A        ┆ Y        ┆ 0.5      │
│ A        ┆ Z        ┆ 0.28125  │
│ B        ┆ X        ┆ 0.140625 │
│ B        ┆ Y        ┆ 0.25     │
│ …        ┆ …        ┆ …        │
│ C        ┆ X        ┆ 0.140625 │
│ C        ┆ Y        ┆ 0.25     │
│ C        ┆ Z        ┆ 0.609375 │
│ B        ┆ null     ┆ 1.0      │
│ C        ┆ null     ┆ 1.0      │
└──────────┴──────────┴──────────┘


In [6]:
import shutil
import tempfile

temp_dir = tempfile.TemporaryDirectory(delete=False)
region_ = create_fake_regions(temp_dir.name)
region_
region_.region_a.geometry
region_.region_b.geometry
region_.region_c.geometry
shutil.rmtree(temp_dir.name)

/home/andre/git/private/electoralyze/packages/electoralyze/electoralyze/common/testing/region_fixture.py:172: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  region_a_gdf.pipe(to_geopandas).to_file(region_a_shape, driver="ESRI Shapefile")
/home/andre/git/private/electoralyze/.pixi/envs/default/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'region_a_name' to 'region_a_n'
  ogr_write(
/home/andre/git/private/electoralyze/packages/electoralyze/electoralyze/common/testing/region_fixture.py:174: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  region_b_gdf.pipe(to_geopandas).to_file(region_b_shape, driver="ESRI Shapefile")
/home/andre/git/private/electoralyze/.pixi/envs/default/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'region_b_name' to 'region_b_n'
  ogr_write(
/home/andre/git/

Loading raw...
Extracting...
Transforming...
Saving...
Done!
Loading raw...
Extracting...
Transforming...
Saving...
Done!
Loading raw...
Extracting...
Transforming...
Saving...
Done!


In [9]:
RegionMocked

electoralyze.common.testing.region_fixture.RegionMocked

In [12]:
data_by_from = pl.DataFrame({"region_b": ["A"], "data": [1]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.region_b,
    region_to=region_.region_a,
    mapping="intersection_area",
    redistribute_with_full=False,
)

shape: (1, 2)
┌──────────┬──────┐
│ region_b ┆ data │
│ ---      ┆ ---  │
│ str      ┆ i64  │
╞══════════╪══════╡
│ A        ┆ 1    │
└──────────┴──────┘


region_a,data
str,f64
"""N""",0.125
"""O""",0.375
"""M""",0.125
"""P""",0.375


In [13]:
data_by_from = pl.DataFrame({"region_c": ["X", "Y", "Z"], "data": [1, 1, 1]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.region_c,
    region_through=region_.region_a,
    region_to=region_.region_b,
    mapping="intersection_area",
    redistribute_with_full=False,
)

shape: (3, 2)
┌──────────┬──────┐
│ region_c ┆ data │
│ ---      ┆ ---  │
│ str      ┆ i64  │
╞══════════╪══════╡
│ X        ┆ 1    │
│ Y        ┆ 1    │
│ Z        ┆ 1    │
└──────────┴──────┘


region_b,data
str,f64
"""C""",0.75
"""B""",0.75
"""A""",1.5
